# Подгонка (*)ARCH (пакет ['arch](https://arch.readthedocs.io/en/latest/index.html))

In [ ]:
import numpy as np
import pandas as pd

from arch import arch_model

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y` - первая разность ставки

In [ ]:
rate = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')
y = rate.diff().dropna()

y.plot()
plt.show()

Подгоним модель AR(2)-GARCH(1,1)

Спецификация

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+\phi_2y_{t-2}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

где $\lambda=2$

In [ ]:
am = arch_model(y, mean='ARX', lags=2, vol='GARCH', p=1, q=1, power=2)

res = am.fit()

res.summary()

In [ ]:
res.plot(annualize='W')
plt.show()

In [ ]:
res.hedgehog_plot(plot_type='volatility')
plt.show()

In [ ]:
res.hedgehog_plot(plot_type='mean')
plt.show()

In [ ]:
res.arch_lm_test(lags=5)